In [2]:
import pandas as pd
import numpy as np
import math
import json
import pprint
import requests
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float

In [3]:
USER_01_MAPPING = 'resources/raw_data_next_gen/admin_users.csv'
PROJECT_01_MAPPING = 'resources/raw_data_next_gen/admin_projects.csv'
ISSUE_01_MAPPING = 'resources/raw_data_next_gen/issues_issues.csv'
COMPANIES_01_MAPPING = 'resources/raw_data_next_gen/admin_companies.csv'
PROJECT_COMPANIES_01_MAPPING = 'resources/raw_data_next_gen/admin_project_companies.csv'
BUSINESS_01_MAPPING = 'resources/raw_data_next_gen/admin_business_units.csv'
ISSUETYPES_01_MAPPING = 'resources/raw_data_next_gen/issues_issue_types.csv'
ISSUESUBTYPES_01_MAPPING = 'resources/raw_data_next_gen/issues_issue_subtypes.csv'
CLASSIGN_01_MAPPING = 'resources/raw_data_next_gen/checklists_checklist_assignees.csv'

##############

USER_02_MAPPING = 'resources/raw_data_old_gen/20200330085907_FieldDataHarvest_Users.csv'
PROJECT_02_MAPPING = 'resources/raw_data_old_gen/20200330085907_FieldDataHarvest_Projects.csv'
ISSUE_02_MAPPING = 'resources/raw_data_old_gen/20200330085907_FieldDataHarvest_Issues.csv'

In [4]:
U1_DF = pd.read_csv(USER_01_MAPPING, encoding='utf-8', low_memory=False)
P1_DF = pd.read_csv(PROJECT_01_MAPPING, encoding='utf-8', low_memory=False)
I1_DF = pd.read_csv(ISSUE_01_MAPPING, encoding='latin1', low_memory=False)
C1_DF = pd.read_csv(COMPANIES_01_MAPPING, encoding='latin1', low_memory=False)
PC1_DF = pd.read_csv(PROJECT_COMPANIES_01_MAPPING, encoding='latin1', low_memory=False)
B1_DF = pd.read_csv(BUSINESS_01_MAPPING, encoding='latin1', low_memory=False)
IT1_DF = pd.read_csv(ISSUETYPES_01_MAPPING, encoding='latin1', low_memory=False)
IST1_DF = pd.read_csv(ISSUESUBTYPES_01_MAPPING, encoding='latin1', low_memory=False)
CA_DF = pd.read_csv(CLASSIGN_01_MAPPING, encoding='latin1', low_memory=False)

U2_DF = pd.read_csv(USER_02_MAPPING, encoding='utf-8', low_memory=False)
P2_DF = pd.read_csv(PROJECT_02_MAPPING, encoding='utf-8', low_memory=False)
I2_DF = pd.read_csv(ISSUE_02_MAPPING, encoding='latin1', low_memory=False)


In [5]:
U1_DF['domain']= U1_DF['email'].str.split('@').str[1]
len(U1_DF['domain'].unique())
domain_A = U1_DF['domain'].unique()

In [6]:
#domain_A[1].split('.')[0]

In [7]:
# temp_DF = U1_DF.loc[U1_DF['domain']==domain_A[16]]
# temp_DF
# LIST_D=[]
# LIST_D = temp_DF.loc[temp_DF['default_company_id'].notnull()]['default_company_id'] 
#res = max(set(LIST_D), key = LIST_D.count) 

In [8]:
# x = LIST_D.value_counts().idxmax()

In [9]:

# r = 0
# #for index, row in U1_DF.iterrows():

# for i in range (1, len(domain_A)):
#     LIST_D=[]
#     r = r + 1
#     temp_DF = U1_DF.loc[U1_DF['domain']==domain_A[i]]
        
#         #LIST_D = temp_DF.loc[temp_DF['default_company_id'].notnull()]
        
#     LIST_D = temp_DF.loc[temp_DF['default_company_id'].notnull()]['default_company_id']
#     if len(LIST_D) > 0:
#         comp_id = LIST_D.value_counts().idxmax()
# #         print("LIST of DOMAINS....",LIST_D)
#         for index, row in temp_DF.iterrows():
#             rowIndex = (index)
        
#             if domain_A[i]>'':
#                 d_value = str(domain_A[i])
#                 if (d_value.split('.')[0]) not in ['gmail','outlook','yahoo','msn','swbell','comcast','icloud','charter']:
        
# #                     print (r,'.....', d_value, '----------GROUP----------------------------')
# #                     for index, row in temp_DF.iterrows():
#                     x1 = (row['domain'])
#                     x2 = (row['default_company_id'])
# #                     print(x2)
                    
#                     if type(x2) == float:
#                         U1_DF.loc[rowIndex, 'default_company_id'] = comp_id
#                         print(rowIndex, x1,'.......',x2)
        
# print ('NEW GROUP-----------------------------------------------------')

In [10]:
#######################################################
P1_DF.rename(columns = {"id" : "project_id"}, inplace=True)
PC1_DF.rename(columns = {"company_id" : "default_company_id"}, inplace=True)
I1_DF.rename(columns = {"bim360_project_id":"project_id"}, inplace=True)
C1_DF.rename(columns = {"id" : "default_company_id"}, inplace=True)

In [11]:
C1_DF.to_csv('cleaned_companies.csv')

In [12]:
U1_DF.count()

id                             9341
autodesk_id                    6769
bim360_account_id              9341
email                          9303
name                           9341
first_name                     9341
last_name                      9341
address_line1                   253
address_line2                    55
city                            314
state_or_province               436
postal_code                     271
country                        9327
last_sign_in                   3101
phone                           324
job_title                       444
access_level_account_admin     9341
access_level_project_admin     9341
access_level_project_member    9341
access_level_executive         9341
default_role_id                 919
default_company_id             3617
domain                         9303
dtype: int64

In [13]:
MERGED_PI_NEW = pd.merge(P1_DF, I1_DF, on ='project_id', how='inner')


In [14]:
# Working_OLD_DataFrame
W_NEW_PI_DF = MERGED_PI_NEW[['project_id',
                   'name',
                   'title',
                   'description',
                   'type_id',
                   'subtype_id',
                   'assignee_id',
                   'job_number',
                   'type',
                   'value',
                   'city',
                   'state_or_province',
                   'postal_code']].copy()

In [15]:
W_NEW_PI_DF.rename(columns = {"type_id" : "issue_type_id"}, inplace=True)
W_NEW_PI_DF.head()

,project_id,name,title,description,issue_type_id,subtype_id,assignee_id,job_number,type,value,city,state_or_province,postal_code
0,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_043_044_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,Springfield,MO,65804
1,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_007_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,Springfield,MO,65804
2,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_019_020_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,Springfield,MO,65804
3,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_021_022_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,Springfield,MO,65804
4,373096cf-bb05-4c80-8cfe-aa4f978ef5d0,00000_Mercy_Hospital_Springfield_Pharmacy,Scan_041_042_High_Low,NaN,fcad3025-f61e-4b23-9aca-fe550600b014,51ee74cd-a8fc-4acf-b665-1e6097c123bd,200810021927646,Unknown,Hospital,1000000.0,Springfield,MO,65804


In [16]:
MERGED_PIIT_NEW = pd.merge(W_NEW_PI_DF, IT1_DF, on ='issue_type_id', how='inner')

In [17]:
#MERGED_PIIT_NEW.head()

In [18]:
MERGED_PIIT_NEW.drop(['is_active','created_by','created_at','updated_by','updated_at','bim360_account_id','bim360_project_id','deleted_by','deleted_at'],inplace=True, axis=1)

In [19]:
MERGED_PIIT_NEW.rename(columns = {"subtype_id" : "issue_subtype_id"}, inplace=True)



In [20]:
#len(IST1_DF['issue_type_id'].unique())

In [21]:
#MERGED_PIIT_NEW.count()
MERGED_PIIST_NEW = pd.merge(MERGED_PIIT_NEW, IST1_DF, on ='issue_subtype_id', how='inner')

In [22]:
MERGED_PIIST_NEW.drop(['issue_type_id_x',
                       'issue_subtype_id',
                       'issue_type_id_y',
                       'is_active',
                       'created_by',
                       'created_at',
                       'updated_by',
                       'updated_at',
                       'deleted_by',
                       'deleted_at',
                       'bim360_account_id',
                       'bim360_project_id'],inplace=True, axis=1)

In [23]:
MERGED_PIIST_NEW.count()

project_id           59084
name                 59084
title                59084
description          23200
assignee_id          52446
job_number           57085
type                 59084
value                57593
city                 42841
state_or_province    42592
postal_code          42401
issue_type           59084
issue_subtype        59084
dtype: int64

In [24]:
#MERGED_PUC_NEW.head()
MERGED_PIIST_NEW.to_csv('NEW_PIIST.csv')

#MERGED_PUC_NEW.rename(columns = {"id" : "project_id"}, inplace=True)

In [25]:
OLD_PI_DF = pd.merge(P2_DF, I2_DF, on='project_id', how='inner')


In [26]:
OLD_PI_DF.rename(columns = {"id":"project_id"}, inplace=True)
OLD_PI_DF.drop(['business_unit_x',
                'status_x',
                'checklists',
                'equipment',
                'tasks',
                'dailyUpdates',
                'photos',
                'users',
                'identifier',
                'status_y','priority',
                'full_description',
                'location_detail',
                'source_type',
                'parent_source_type',
                'created_by',
                'area',
                'created_at',
                'updated_at',
                'created_from',
                'due_date',
                'closed_at',
                'root_cause',
                'hq_identifier_y',
                'business_unit_y'],inplace=True, axis=1)

In [27]:
OLD_PI_DF.count()

hq_identifier_x     101605
project_id          101605
name                101605
files               101605
issues              101605
issue_id            101605
description         101605
issue_type          101605
company_assigned     96657
dtype: int64

In [28]:
OLD_PI_DF.rename(columns = {"hq_identifier_x":"project_id","project_id":"old_project_id"}, inplace=True)

In [29]:
MERGED_PI_OLD2NEW = pd.merge(OLD_PI_DF, P1_DF, on ='project_id', how='inner')

In [30]:
MERGED_PI_OLD2NEW.count()
MERGED_PI_OLD2NEW.drop(['files',
                        'bim360_account_id',
                        'address_line1',
                        'address_line2',
                        'start_date',
                        'end_date',
                        'currency',
                        'timezone',
                        'construction_type',
                        'contract_type',
                        'business_unit_id',
                        'last_sign_in',
                        'created_at',
                        'country',
                        ],inplace=True, axis=1)

In [31]:
###################################################
#### TO CLEAN
MERGED_PI_OLD2NEW.head()
MERGED_PI_OLD2NEW=MERGED_PI_OLD2NEW.reindex(columns = MERGED_PI_OLD2NEW.columns.tolist() + ['issue_subtype','title'])


In [32]:
PI_OLD2NEW_DF = MERGED_PI_OLD2NEW[['project_id',
                                  'name_x',
                                  'job_number',
                                  'type',
                                  'value',
                                  'city',
                                  'state_or_province',
                                  'postal_code',
                                  'issue_type',
                                  'issue_subtype',
                                  'title',
                                  'description',
                                  ]].copy()

In [33]:
PI_OLD2NEW_DF.rename(columns = {"name_x":"name"}, inplace=True)

In [34]:
PIIST_NEW_DF = MERGED_PIIST_NEW [['project_id',
                                  'name',
                                  'job_number',
                                  'type',
                                  'value',
                                  'city',
                                  'state_or_province',
                                  'postal_code',
                                  'issue_type',
                                  'issue_subtype',
                                  'title',
                                  'description',
                                   ]].copy()

In [35]:
PIIST_NEW_DF.count()

project_id           59084
name                 59084
job_number           57085
type                 59084
value                57593
city                 42841
state_or_province    42592
postal_code          42401
issue_type           59084
issue_subtype        59084
title                59084
description          23200
dtype: int64

In [84]:
PI_NEWandOLD_DF = pd.concat([PI_OLD2NEW_DF, PIIST_NEW_DF], axis = 0)

In [85]:
PI_NEWandOLD_DF.count()

project_id           160689
name                 160689
job_number           108330
type                 160689
value                158952
city                 124587
state_or_province    125271
postal_code          120071
issue_type           160689
issue_subtype         59084
title                 59084
description          124805
dtype: int64

In [86]:
indexScanns = PI_NEWandOLD_DF[PI_NEWandOLD_DF['issue_subtype'] == 'Scanning' ].index


In [87]:
#indexFload = PI_NEWandOLD_DF.select_dtypes(include=['float64'])
print(PI_NEWandOLD_DF['description'].dtypes)

object


In [88]:
len(indexScanns)

702

In [89]:
PI_NEWandOLD_DF.dtypes

project_id            object
name                  object
job_number            object
type                  object
value                float64
city                  object
state_or_province     object
postal_code           object
issue_type            object
issue_subtype         object
title                 object
description           object
dtype: object

In [90]:
PI_NEWandOLD_DF.drop(indexScanns, inplace=True)

In [91]:
PI_NEWandOLD_DF['description'].fillna('', inplace=True)

In [92]:
PI_NEWandOLD_DF['title'].fillna('', inplace=True)


In [93]:
TEMP_DF = PI_NEWandOLD_DF

In [97]:
def description(row):
    if len(row['description']) > len(row['title']):
        return row['description']
    if len(row['description']) < len(row['title']):
        return row['title']

In [98]:
PI_NEWandOLD_DF['description2']=PI_NEWandOLD_DF.apply (lambda row: description(row), axis =1)

In [94]:
# for index, row in PI_NEWandOLD_DF.iterrows():
#     rowIndex = (index)
#     ld = row['description']
#     lt = row['title']
#     print(index)
#     if len(ld) > len(lt):
#         print(ld)
#         TEMP_DF.loc[rowIndex, 'new_description'] = ld
#     else:
#         print(lt)
#         TEMP_DF.loc[rowIndex, 'new_description'] = lt

0
Remove nails around dock angles. 
1
Grind in a 3/4" chamfer at all exterior corners on the tilt panels on all 4 corners and on the inside parapet corners at the truckers lounges and upper parapets per detail A2/A08-20.
2
Install missing switch and data outlets
3
Install cover plate on light switch
4
Install base
5
Repaint north and south walls
6
Complete striping and traffic signage
7
Clean floor, typ.
8
Touch up yellow paint at railing door x118
9
Fix weatherstripping at door x119
10
Fill nail holes in all door windows, offices and mezz 200
11
Paint east wall
12
Paint above door
13
Verify no base heater in women's

Clayco response - no base heater in women's 144
14
Patch top of SE drive in ramp so it is level along the length and the chamfer is consistent along the length. 
15
Clean haze from tile walls
16
Remove stick pin insulation, fur out drywall to deck per drawings, paint
17
Replace thermax in electrical room as shown
18
Door sa2a  & sa2b add screws to door frame
19
Touch up p

186
Cut back expansion material SE ramp
187
Finish installing faucet
188
Install light strobes
189
Complete air TAB
190
Install gravel at cooling tower area
191
Remove haze from wall tile
192
Install floor filler
193
Remove blue line and mortar from exterior rack storage wall, grind smooth, remove tie wires, remove for work
194
Install wall heater
195
Caulk tilt up joint on south wall about 20' from sw corner
196
Repaint sw corner 
197
Remove tape from column
198
Install overhead door
199
Clean mortar and drywall globs off 48" high wall along offices and touch up paint, check entire wall
200
Add all stickers on supply and return for process water, compressed air
201
Seal after connection with spray foam, put architectural cover over hole
202
Touch up paint on wall after wall is repaired 
203
Oh 109 adjust leveler
204
Clean concrete splatter off walls at top of drive in ramps before painting
205
Touch up paint on door and frame
206
Clean floors
207
Seal or trim piece around all panel bo

390
Touch up paint at door x115
391
Seal all penetrations in wall
392
Install guy wire material around tree plantings per L2 drawing
393
Adjust 2nd floor platform door, door does not latch
394
Caulk around exterior door
395
Finish life safety and electrical work
396
Add door frame screws to both doors
397
Touch up scratches on door
398
Touch up paint on walls
399
Joint fill floor
400
Sawcut the cold joints in the slabs. 
401
Add cover plate to dimmer switch and touch up around thermostat
402
Complete floor and install thresholds at exterior doors. 
403
Install base
404
Install zone maps and riser identification
405
Finish plumbing in shower
406
Touch up wall paint at center break area
407
Touch up paint
408
Provide full bed of sealant under subsills per details on A09-20. Occurs along north wall and break room windows. Check east wall also. 
409
Install switch and add data
410
Seal all wall penetrations
411
Caulk around columns
412
Paint patched drywall, typ.
413
Repaint scratch outsid

553
Touch up paint at ceiling south wall
554
Repaint door frame after rework 
555
Conduit fittings not tight. 
556
Repaint around perimeter of window seal due to rework
 Remove 
557
Fire caulk waste penetration to level 3 above shower is loose.
558
Complete electrical connection for theFCU and install light fixtures. 
559
Fire caulk block wall to concrete foundation wall.
560
ERU-2  service area thermostat should be located in the corridor.  It should not be in the always-hot cabinet.
561
Remove wall paint and adhesive from window system
562
Finish painting bulkhead and install grill.
563
Install door and hardware
564
Install devices/plates after paint
565
<no description>
566
Touch up paint on wall and ceiling around sprinkler head 
Touch up paint on wall above door frame 
Touch up paint on wall at window sill 
Touch up paint on window sill jambs both sides 
Touch up paint on door frame top 
567
Paint ceiling.
568
Replace smoke seal due to paint
569
Exterior Canopy (south side):  the 

680
REPOSTED Aqua-Therm piping going through foundation wall in Basement. See attached picture. Appears the pipe on the right is sloping left. Has Aqua Therm reviewed? Any concern? Test is for pressure testing.
681
Install door and hardware
682
Touch up paint in the following locations indicated in the photos 
683
Clean bugs out of light
684
Paint walls above FRP.
685
Adjust door to completely close and latch
686
Antares - Fire extinguisher missing tag and tamper seal
687
Fire caulk sprinkler penetration into RA Apartment 
688
POWERED INDUSTRIAL VEHICLE USE
Subcontractor skid steer had a missing FE - corrected and addressed with foreman
689
Install pipe shield HHWS
690
Install door and hardware
691
Install missing electrical device in ceiling 
692
Wall needs repainted under electric panel
693
Clean floor sink rust and dirt.
694
Room should be R011 not R013
695
Caulk walls between cmu and drywall
696
Install L-B covers
697
Paint is chipped off of storefront.
698
Remove plywood from unde

...

36714
Sill caulking at drywall to sill cap reveal
Secure sill cap at seam cap to cap
Caulk if needed 
36715
Caulk metal tile trim to drywall
36716
Plumbing Fixtures 
36717
Drywall 
36718
Tape off of floor drain
36719
Install blank cover plate on empty outlet box

36720
Repair Drywall Where Indicated by Tape/Sticker
36721
Repair Paint where indicated by tape/sticker
36722
Clean up door frame and sidelight
36723
Missing receptacle Labels all areas.
36724
Install Base
36725
Repair scratches on Sill/Mullion as Indicated by Tape/Sticker
36726
Repair Paint where indicated by tape/sticker
36727
Repair Carpet Where Indicated by Tape/Sticker
36728
Repair scratches on Sill/Mullion as Indicated by Tape/Sticker
36729
Repair Paint where indicated by tape/sticker
36730
Repair Ceramic Tile Damage Where Indicated by Tape/Sticker.
36731
Remove debris from atop fixture lens
36732
Repair Paint where indicated by tape/sticker
36733
Gap 
36734
Repair or replace door.
36735
Sprinkler Heads and Escutcheons 


36897
repair paint where indicated
36898
Doors have gaps an inch or greater. 
36899
Light Fixtures 
36900
Work needs completed, fire protection, etc. This will likely hold off drywall. 
36901
Drywall 
36902
Repair/ Replace Acoustical Ceiling Tile as Indicated by Tape/Sticker or as Noted.  
36903
Mechanical Room door does not latch
36904
repair ceiling grid where indicated
36905
Replace damaged/dirty/marked ceiling tiles where indicated with blue tape.
36906
Clean door frames and sidelites.
36907
Repair Cabinets Where Indicated by Tape/Sticker or as Noted.  
36908
Patch hole in soffit west side
36909
Repair Paint where indicated by tape/sticker
36910
Repair/ Replace Acoustical Ceiling Tile as Indicated by Tape/Sticker or as Noted.  
36911
Plumbing Fixtures 
36912
Bent Acoustical Ceiling Grid
36913
Repair/ Replace Acoustical Ceiling Tile as Indicated by Tape/Sticker or as Noted.  
36914
Clean light fixtures. 
Clean door frame and sidelight 
36915
Frame to door 3004 dented. 
36916
trim ou

37096
Repair Drywall Where Indicated by Tape/Sticker
37097
Door stop missing.
37098
Cabinets 
37099
Touch up paint as marked. 
37100
Fire seal wall above ceilings. Along deck as well.
37101
Repair Vinyl Planks/Tile Where Indicated by Tape/Sticker
37102
Acoustical Ceiling Grid 
37103
West Side - Remove spots/stains from expose concrete
37104
Flow meter should be mounted to something. (area behind shaft elevator)
37105
Outlets 
37106
Door hits frame when closing.
37107
Repair/ Replace Acoustical Ceiling Tile as Indicated by Tape/Sticker or as Noted.  
37108
Repair Paint where indicated by tape/sticker
37109
Paint on column cover 
37110
Carpet does not terminate under base exposing concrete and carpet is frayed at door. 
37111
General: paint touch up. 
37112
<no description>
37113
Complete Final Clean Items listed.
37114
Repair/ Replace Acoustical Ceiling Tile as Indicated by Tape/Sticker or as Noted.  
37115
Sprinkler Heads and Escutcheons 
37116
Finish base. 
37117
Repair Carpet Where I

37286
closer cover
37287
paint caulk around mop basin
37288
Clean and buff cw sills.
37289
Stainless steel shoes that attach partition to ceiling are warped on some and not sealed to ceiling.
37290
CX: VAV 11-12: Move sprinkler pipe for access to the VAV control panel.
37291
Drywall 
37292
Patch holes on north wall
37293
Door hitting the base of door stop not rubber stop. 
37294
trim out aluminum door
37295

KeyboardInterrupt: 

In [83]:
PI_NEWandOLD_DF.head(150000)

,project_id,name,job_number,type,value,city,state_or_province,postal_code,issue_type,issue_subtype,title,description,new_description
0,01fa4f3a-9969-4069-a5b3-72fb630a0143,Gestamp,102601,Manufacturing / Factory,20549347.0,McCalla,AL,35111,Punch List : Pre-Punch List,NaN,,Remove nails around dock angles.,Remove nails around dock angles.
1,01fa4f3a-9969-4069-a5b3-72fb630a0143,Gestamp,102601,Manufacturing / Factory,20549347.0,McCalla,AL,35111,Punch List : Pre-Punch List,NaN,,"Grind in a 3/4"" chamfer at all exterior corner...","Grind in a 3/4"" chamfer at all exterior corner..."
2,01fa4f3a-9969-4069-a5b3-72fb630a0143,Gestamp,102601,Manufacturing / Factory,20549347.0,McCalla,AL,35111,Punch List : Architect Punch List,NaN,,Install missing switch and data outlets,Install missing switch and data outlets
3,01fa4f3a-9969-4069-a5b3-72fb630a0143,Gestamp,102601,Manufacturing / Factory,20549347.0,McCalla,AL,35111,Punch List : Architect Punch List,NaN,,Install cover plate on light switch,Install cover plate on light switch
4,01fa4f3a-9969-4069-a5b3-72fb630a0143,Gestamp,102601,Manufacturing / Factory,20549347.0,McCalla,AL,35111,Punch List : Architect Punch List,NaN,,Install base,Install base
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49112,8d1b3747-e7f2-4e2d-bc86-e394242c840c,10103255 - Macy's Chicago,10103255,Office,120000000.0,Chicago,IL,60602,Safety,Safety,Working on roof without tie off,Worker was 15-20ft above street level without ...,Install Base at Millwork Toekick
49113,8d1b3747-e7f2-4e2d-bc86-e394242c840c,10103255 - Macy's Chicago,10103255,Office,120000000.0,Chicago,IL,60602,Work List,Work List,Demo opening in wall for new MEP pipe,GLP needs this opening demoed for new piping t...,Install concealed sprinkler head plate
49114,8d1b3747-e7f2-4e2d-bc86-e394242c840c,10103255 - Macy's Chicago,10103255,Office,120000000.0,Chicago,IL,60602,Work List,Work List,Demo wall for shaft framing to be installed,This small section of wall needs to be demoed ...,Caulk exterior of single man door leaving b2 s...
49115,8d1b3747-e7f2-4e2d-bc86-e394242c840c,10103255 - Macy's Chicago,10103255,Office,120000000.0,Chicago,IL,60602,Work List,Work List,Cut off flush with the floor,This either needs to be unbolted or cut off flush,Touch up wall in pre function area in b2 south


In [44]:
PI_NEWandOLD_DF.to_csv('PI_NEWandOLD.csv')

In [ ]:
################################################################################
### END SECTION
#################################################################################

In [23]:
I1_DF.rename(columns = {"bim360_project_id" : "project_id"}, inplace=True)
MERGED_NEW.to_csv('new_clean.csv')
MERGED_OLD.rename(columns = {"project_id" : "project_id_classic"}, inplace=True)
MERGED_OLD.rename(columns = {"hq_identifier_x" : "project_id"}, inplace=True)
MERGED_OLD_NEXT = pd.merge(MERGED_OLD, P1_DF, on='project_id', how='inner')
MERGED_OLD_NEXT.to_csv('old_next_clean.csv')
### FINAL MERGE PROJECT ISSUES
OLD_MAPPING = 'old_next_clean.csv'
NEW_MAPPING = 'new_clean.csv'
##################################
OLD_DF = pd.read_csv(OLD_MAPPING, encoding='utf-8', low_memory=False)
NEW_DF = pd.read_csv(NEW_MAPPING, encoding='utf-8', low_memory=False)
# Working_OLD_DataFrame
W_OLD_DF = OLD_DF[['project_id',
                   'bim360_account_id'
                   'business_unit_x',
                   'name_x',
                   'issues',
                   'users',
                   'issue_id',
                   'description',
                   'issue_type',
                   'status_x',
                   'company_assigned',
                   'job_number',
                   'identifier',
                   'created_at_x',
                   'due_date',
                   'closed_at',
                   'company_assigned',
                   'created_by',
                   'root_cause',
                   'area',
                   'location_detail',
                   'type',
                   'value',
                   'city',
                   'state_or_province',
                   'postal_code']].copy()
W_OLD_DF.head()
W_OLD_DF.rename(columns = {"name_x" : "name",
                           'business_unit_x' : 'business_unit',
                           'status_x' : 'status',
                           'created_at_x' : 'created_at'}, inplace=True)
# W_NEW_DF = NEW_DF[['project_id',
#                    'bim360_account_id_x'
#                    'business_unit_x',
#                    'name',
#                    'issues',
#                    'users',
#                    'issue_id',
#                    'title',
#                    'description',
#                    'issue_type',
#                    'status_x',
#                    'company_assigned',
#                    'job_number',
#                    'identifier',
#                    'created_at_x',
#                    'due_date',
#                    'closed_at',
#                    'company_assigned',
#                    'created_by',
#                    'root_cause',
#                    'area',
#                    'location_detail',
#                    'type',
#                    'value',
#                    'city',
#                    'state_or_province',
#                    'postal_code']].copy()